In [2]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import display, Image
import pandas as pd

In [3]:
source_file = 'training_elephants_split.csv'

df_coords = pd.read_csv(source_file, header=None, skiprows=1, names=['fname', 'x', 'y'])
df_coords['bbox_x'] = np.nan
df_coords['bbox_y'] = np.nan
df_coords['bbox_w'] = np.nan
df_coords['bbox_h'] = np.nan
df_coords['area'] = np.nan

display(df_coords[:10])

,fname,x,y,bbox_x,bbox_y,bbox_w,bbox_h,area
0,8f30778719e5af075797f994273603c94f170e29_00,74,168,NaN,NaN,NaN,NaN,NaN
1,8f30778719e5af075797f994273603c94f170e29_02,804,724,NaN,NaN,NaN,NaN,NaN
2,8f30778719e5af075797f994273603c94f170e29_02,756,754,NaN,NaN,NaN,NaN,NaN
3,8f30778719e5af075797f994273603c94f170e29_02,654,720,NaN,NaN,NaN,NaN,NaN
4,8f30778719e5af075797f994273603c94f170e29_02,850,758,NaN,NaN,NaN,NaN,NaN
5,8f30778719e5af075797f994273603c94f170e29_02,786,798,NaN,NaN,NaN,NaN,NaN
6,8f30778719e5af075797f994273603c94f170e29_02,474,626,NaN,NaN,NaN,NaN,NaN
7,8f30778719e5af075797f994273603c94f170e29_11,60,768,NaN,NaN,NaN,NaN,NaN
8,4808725b78d40aa66681ad501c830d735f6b1314_10,570,13,NaN,NaN,NaN,NaN,NaN
9,fcea4eb8b7a5366c5b751bd154e2109fc0f59ea8_00,135,106,NaN,NaN,NaN,NaN,NaN


In [11]:
for i in range(df_coords.shape[0]):
    image_name, center_x, center_y = df_coords.iloc[i, 0:3].values
    image = cv2.imread("split_gsd_training_images/" + image_name + ".jpg")

    h, w = image.shape[0:2]
    # Replace with your actual center coordinates
    # Define the ROI around the center point
    roi_size = 25
    x1, y1 = center_x - roi_size, center_y - roi_size
    x2, y2 = center_x + roi_size, center_y + roi_size
    
    if x1 < 0: x1 = 0
    if y1 < 0: y1 = 0
    if x2 > w: x2 = w - 1
    if y2 > h: y2 = h - 1

    roi = image[y1:y2, x1:x2]

    # Convert ROI to grayscale
    try:
        gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    except:
        print(image_name)
        print(roi)
        print(i)
    # Apply Canny edge detection
    edges = cv2.Canny(gray_roi, 50, 150)

    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw the contours on a black background with the correct offset
    boxes = []
    for contour in contours:
        adjusted_contour = contour + (x1, y1)
        x, y, w, h = cv2.boundingRect(adjusted_contour)
        
        area = 0.5 * w * h
        boxes.append([x, y, w, h, area])

    boxes = np.array(boxes)

    try:
        boxes = boxes[boxes[:, 4].argsort()[::-1]]
    except:
        continue

    boxes = boxes.astype(int)
    x, y, w, h, area = boxes[0]

    df_coords.iat[i, 3] = int(x)
    df_coords.iat[i, 4] = int(y)
    df_coords.iat[i, 5] = int(w)
    df_coords.iat[i, 6] = int(h)
    df_coords.iat[i, 7] = area

    if i % 100 == 0:
        print(str(i) + " items completed.")

df_coords.to_csv('training_elephants_split_bboxes.csv', index=False)   


0 items completed.
100 items completed.
200 items completed.
300 items completed.
400 items completed.
500 items completed.
600 items completed.
700 items completed.
800 items completed.
900 items completed.
1000 items completed.
1100 items completed.
1200 items completed.
1300 items completed.
1400 items completed.
1500 items completed.
1600 items completed.
1700 items completed.
1800 items completed.
1900 items completed.
2000 items completed.
2100 items completed.
2200 items completed.
2300 items completed.
2400 items completed.
2500 items completed.
2600 items completed.
2700 items completed.
2800 items completed.
2900 items completed.
3000 items completed.
3100 items completed.
3200 items completed.
3300 items completed.
3400 items completed.
3500 items completed.
3600 items completed.
3700 items completed.
3800 items completed.
3900 items completed.
4000 items completed.
4100 items completed.
4200 items completed.
4300 items completed.
4500 items completed.
4600 items completed.


In [37]:
rand_image = df_coords.sample().values[0]
df_coords_rand = df_coords[df_coords.fname == rand_image[0]].values
image_with_bounding_box = cv2.imread("split_gsd_training_images/" + str(df_coords_rand[0,0]) + ".jpg")
print(len(df_coords_rand))

for img in df_coords_rand:
    x = int(img[3])
    y = int(img[4])
    w = int(img[5])
    h = int(img[6])

    # plt.imshow(image_with_bounding_box)
    cv2.rectangle(image_with_bounding_box, (x, y), (x + w, y + h), (0, 255, 0), 2)
cv2.imwrite('test.jpg', image_with_bounding_box) 

9


True